[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/joshmaglione/CS102-Jupyter/main?labpath=.%2FWeek07.ipynb) 

<a href="https://colab.research.google.com/github/joshmaglione/CS102-Jupyter/blob/main/Week07.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> 

[View on GitHub](https://github.com/joshmaglione/CS102-Jupyter/blob/main/Week07.ipynb)

# Week 7: Working with DataFrames

Last time we discussed the three main data structures `pandas` brings.

Now we will discuss how to manipulate the primary object: DataFrames.

Let's load in UN data about Ireland from two files sources: 
- `data/01_below_poverty.csv`
- `data/07_renewable_energy.csv`

(This was obtained from the [United Nations' SDG Country Profile Page](https://unstats.un.org/sdgs/dataportal/countryprofiles/IRL))

This data has lots of information that we don't need, so our goal is to produce one DataFrame with the information we want. 

In [ ]:
import numpy as np
import pandas as pd
print(f"pandas version: {pd.__version__}")

You can [read more](https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html#copy-on-write) about their upcoming changes to `pandas 3.0`.

## Below Poverty Data Set

We'll work through some of the basics with the below poverty data set.

In [ ]:
df1 = pd.read_csv("data/01_below_poverty.csv")
df1.head()

Most of the columns look irrelevant. Let's look to keep the columns
- "TimePeriod"
- "Value"
- "Time_Detail"
- "Age"
- "Location"
- "Sex"

In [ ]:
df1 = df1[[
	"TimePeriod",
	"Value",
	"Time_Detail",
	"Age",
	"Location",
	"Sex"
]]
df1.head()

It's weird that `"TimePeriod"` is a float. Let's change this to an int.

In [ ]:
# df1.astype({"TimePeriod" : "i"})

We need to clean our data before we can manipulate it. Let's remove the rows where `"TimePeriod"` is empty.

In [ ]:
df1 = df1.dropna(subset="TimePeriod")
df1 = df1.astype({"TimePeriod" : "i"})
df1

We are not going to span the entirity of this data set. Let's just take the years 2005 to 2020.

We'll do this with the [`query`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html) command. 

At a basic level it takes a string that appropriately masks part of the dataframe.

By default, `query` is a copy, unlike slicing in `ndarray` which yields a view.

In [ ]:
df1_check = df1.query("2005 <= TimePeriod <= 2020")
df1.at[22, "TimePeriod"] = 1000     # Index 22 is the first row with 2005
df1.loc[22]

In [ ]:
df1 = df1_check
df1

The `"Time_Detail"` column looks irrelevant. 

Let's look at all the values and the number of time they arise with [`value_counts`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html).

In [ ]:
df1["Time_Detail"].value_counts()

In [ ]:
df1 = df1.drop(["Time_Detail"], axis=1)
df1

The last four columns are potentially irrelevant. Let's see what the possibilities are for 
- "Age"
- "Location"
- "Sex"

We want the most inclusive options.

Let's look at the rows for one particular year. 

In [ ]:
df1.query("2011 == TimePeriod")

The data set has lots of granularity. Let's just take the coarse, general information.

We'll do this by chaining three queries. 

In comments, we have an equivalent version, but it is generally considered good practice to chain as follows.

*This is just for aesthetic and reabability reasons.*

In [ ]:
# df1 = df1[
# 	(df1["Age"] == "ALLAGE") & 
# 	(df1["Location"] == "ALLAREA") &
# 	(df1["Sex"] == "BOTHSEX")
# ]
df1 = (df1
    .query("Age == 'ALLAGE'")
    .query("Location == 'ALLAREA'")
    .query("Sex == 'BOTHSEX'")
)
df1

Now that the last three columns are constant, we will drop them.

In [ ]:
df1 = df1.drop(["Age", "Location", "Sex"], axis=1)
df1

The current index is also irrelevant. Let's convert `"TimePeriod"` to our index. 

In [ ]:
df1 = df1.set_index("TimePeriod")
df1

Lastly, let's change `"Value"` to `"Below Poverty (%)"`.

In [ ]:
df1 = df1.rename(columns={"Value" : "Below Poverty (%)"})
df1

## Speedrun: Renewable Energy Data Set

We will basically do the same steps as above, but all at once. See if you can follow along line by line.

In [ ]:
df2 = pd.read_csv("data/07_renewable_energy.csv")
df2 = df2[["TimePeriod", "Value"]]
df2 = df2.dropna()
df2 = df2.astype({"TimePeriod" : "i"})
df2 = df2.set_index("TimePeriod")
df2 = df2.loc["2005":"2020"]
df2 = df2.rename(columns={"Value" : "Renewable Energy Share (%)"})
df2

## From two to one

Because our two DataFrames have the same index, we can concatenate them in `pandas`.

In [ ]:
df = pd.concat([df1, df2], axis=1)		# Merging our two DataFrames
df.index.names = ["Year"]				# Renaming the index
df

We can get some quick statistics on our data.

Using the `describe` method, we get the average and standard deviations.

In [ ]:
df.describe()

#### Detour: visualization

We'll discuss this more later, but we can now plot the DataFrame on a set of axes with `Matplotlib` (working in the background).

In [ ]:
df.plot()

## Ufuncs

All the `NumPy` UFuncs can be applied to DataFrames, provided they are applied to appropriate numerical data. 

First let's generate some random data.

In [ ]:
N = 5

# Generate some random data with N + 1 rows
IDs = np.arange(2*N)
np.random.shuffle(IDs)
ID1 = IDs[:N + 1].copy()
ID1.sort()
df1 = np.floor(pd.DataFrame({
	"Height (cm)" : np.random.normal(168, 8, N + 1),
	"Weight (km)" : np.random.normal(82, 9, N + 1),
}, index=ID1)).astype("i")
df1

In [ ]:
# Generate more random data with N + 2 rows
np.random.shuffle(IDs)
ID2 = IDs[:N + 2].copy()
ID2.sort()
df2 = np.floor(pd.DataFrame({
	"Age (y)" : np.random.normal(40, 5, N + 2),
	"Heart rate (bpm)" : np.random.normal(78, 9, N + 2),
}, index=ID2)).astype("i")
df2

By using Ufuncs on Series, we can compute a BMI column in the first data set.

In [ ]:
df1["BMI"] = np.round(df1["Weight (km)"] / (df1["Height (cm)"]/100)**2).astype("i")
df1

Before we continue to, it will be useful to use the following utility function.

In [ ]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Concatenation

We have already seen a special case of concatenation, but that had the benefit of having indices the same.

We can concatenate the two dataframes to get

In [ ]:
pd.concat([df1, df2], axis=1, sort=True)

We may want to *exclude* those rows that are not complete. We can do this by setting `join="inner"`.

In [ ]:
pd.concat([df1, df2], axis=1, join="inner", sort=True)

The default is `join="outer"`.

Another example taken from the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#joining-logic-of-the-resulting-axis).

In [ ]:
dfa = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)

dfb = pd.DataFrame(
    {
        "B": ["B2", "B3", "B6", "B7"],
        "D": ["D2", "D3", "D6", "D7"],
        "F": ["F2", "F3", "F6", "F7"],
    },
    index=[2, 3, 6, 7],
)

result = pd.concat([dfa, dfb], axis=1)

In [ ]:
display('dfa', 'dfb', 'result')

## Merging

Merging is similar to concatenation, but significantly different. 😅

In [ ]:
display('dfa', 'dfb', 'dfa.merge(dfb)')

Unlike with concatenation, DataFrames are joined by default using `'inner'`, but with the `how` keyword argument.

With `merge`, there are many more options. One can set `how` to be:
- `inner`
- `outer`
- `left`
- `right`
- `cross`

In [ ]:
display('dfa', 'dfb', 'dfa.merge(dfb, how="outer")')

By default, the largest set of compatible columns are choosen. 

We can merge on fewer columns by changing the `on` keyword argument.

In [ ]:
display('dfa', 'dfb', 'dfa.merge(dfb, how="outer", on="B")')

We can change the `_x` and `_y` using the `suffixes` keyword argument. 

In [ ]:
display('dfa', 'dfb', 'dfa.merge(dfb, how="outer", on="B", suffixes=("_left", "_right"))')

Setting `how='left'` pulls only those rows from the left DataFrame. Similarly for `how='right'`.

# Exercises

1. Load the DataFrames associated to `03_road_deaths.csv` and `09_CO2_combustion.csv`.
    - Build a DataFrame (either by concatenation or merging) with the main value from both DataFrames for both sexes (in `03_....csv`) and for the total amount (in `09_....csv`). The time span is up to you, but it should be at least 10 years.
    - Determine the average and standard deviation during the time span you determined.